In [18]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import sys
import time
import warnings
import matplotlib as mpl
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
torch.cuda.is_available()

True

In [19]:
train_all = pd.read_csv('train.csv')
test_all = pd.read_csv('test.csv')

datasets = [train_all, test_all]

In [20]:
train_all.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [21]:
train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [22]:
# for dataset in datasets:
#   dataset.drop(["Alley", "MiscFeature", "PoolQC", "Fence"], axis=1, inplace = True)
#   for col in dataset.columns:
#     if dataset[col].isnull().sum() > 1000:
#       print(col, dataset[col].isnull().sum())


In [23]:
corrmat = train_all.corr()
corrmat["SalePrice"].sort_values(ascending=False)



SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [24]:
train_all = train_all.loc[:, ["SalePrice", "OverallQual", "GrLivArea", "GarageCars", "GarageArea",
                              "TotalBsmtSF", "1stFlrSF", "FullBath", "TotRmsAbvGrd", "YearBuilt", "YearRemodAdd"]]
test_all = test_all.loc[:, ["OverallQual", "GrLivArea", "GarageCars", "GarageArea",
                            "TotalBsmtSF", "1stFlrSF", "FullBath", "TotRmsAbvGrd", "YearBuilt", "YearRemodAdd"]]

train_all.describe()
train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   SalePrice     1460 non-null   int64
 1   OverallQual   1460 non-null   int64
 2   GrLivArea     1460 non-null   int64
 3   GarageCars    1460 non-null   int64
 4   GarageArea    1460 non-null   int64
 5   TotalBsmtSF   1460 non-null   int64
 6   1stFlrSF      1460 non-null   int64
 7   FullBath      1460 non-null   int64
 8   TotRmsAbvGrd  1460 non-null   int64
 9   YearBuilt     1460 non-null   int64
 10  YearRemodAdd  1460 non-null   int64
dtypes: int64(11)
memory usage: 125.6 KB


In [25]:
train, valid = train_test_split(
    train_all, test_size=0.1, random_state=42, shuffle=True)

train_X, train_y = train.drop('SalePrice', axis=1), train['SalePrice']
valid_X, valid_y = valid.drop('SalePrice', axis=1), valid['SalePrice']
num_pip = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

train_X = num_pip.fit_transform(train_X)
train_X

array([[ 0.65007143,  1.03552468, -1.03715877, ..., -0.33702514,
        -1.14967906,  0.10539079],
       [ 0.65007143, -0.17341627,  0.31083987, ..., -0.33702514,
         0.98492863,  0.83043525],
       [-0.80635459, -1.44984196, -1.03715877, ..., -0.95813458,
         0.0325652 ,  0.87877155],
       ...,
       [-0.80635459, -1.26474095,  0.31083987, ..., -0.95813458,
        -0.52571681,  0.25039968],
       [ 0.65007143, -0.18112881, -1.03715877, ...,  0.2840843 ,
        -1.74080119,  0.63709006],
       [ 0.65007143,  0.06760066,  1.6588385 , ...,  0.2840843 ,
         1.18196935,  1.07211674]])

In [26]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
model = keras.Sequential([
    keras.layers.Dense(16, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(32, activation='elu'),
    keras.layers.Dense(64, activation='elu'),
    keras.layers.Dense(128, activation='elu'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(1)
])

In [29]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss='mean_squared_error', metrics=['mean_squared_error'])

In [31]:
model.fit(train_X, train_y, epochs=20, validation_data=(valid_X, valid_y))

Epoch 1/20
42/42 [==============================] - 0s 7ms/step - loss: 3415233536.0000 - mean_squared_error: 3415233536.0000 - val_loss: 2590111166365696.0000 - val_mean_squared_error: 2590111166365696.0000
Epoch 2/20
42/42 [==============================] - 0s 5ms/step - loss: 2888061440.0000 - mean_squared_error: 2888061440.0000 - val_loss: 3873187012018176.0000 - val_mean_squared_error: 3873187012018176.0000
Epoch 3/20
42/42 [==============================] - 0s 5ms/step - loss: 2369686784.0000 - mean_squared_error: 2369686784.0000 - val_loss: 8554357767274496.0000 - val_mean_squared_error: 8554357767274496.0000
Epoch 4/20
42/42 [==============================] - 0s 5ms/step - loss: 1745224064.0000 - mean_squared_error: 1745224064.0000 - val_loss: 9638632461697024.0000 - val_mean_squared_error: 9638632461697024.0000
Epoch 5/20
42/42 [==============================] - 0s 5ms/step - loss: 1841322240.0000 - mean_squared_error: 1841322240.0000 - val_loss: 8309336862359552.0000 - val_me